# Main Notebook

## importing files and loading csv

In [5]:
# important imports
import pandas as pd
import numpy as np

In [6]:
# loading csv
df = pd.read_csv("/Users/ekamsinghahuja/Desktop/untitled folder/Training Data.csv")

# Reduction of Data
## this is done due to massive amount of data is pointing towards not approved 
few of our models are predicting only not approved and sill we are getting accuracy above 80. we have also tried SMOT but that is not good for huge categorical data .we have tried other formats of smot as well, like smotenc .But it does not work . So, we have used data reduction

In [7]:
from sklearn import imblearn.over_sampling.SMOTENC
df1 = df[df['risk_flag']==0]
df2 = df[df['risk_flag']==1]
from sklearn.utils import shuffle
df1 = shuffle(df1)
df1=df1.reset_index()
df1 = df1.iloc[0:40000,:]
df1 = df1.drop('index',axis=1)
df = pd.concat([df1,df2],ignore_index=True)
df=shuffle(df)
df=df.reset_index()
df = df.drop('index',axis=1)

## Creating list of columns required to be Deleted, Encoded and Scaled

In [8]:
drop_cols = ['Id','city']
#onehot encoded columns
target_col = ['car_ownership','house_ownership','married','current_job_years','current_house_years','experience','state','profession']
#label encoded columns
object_cols=[]

In [9]:
df = df.drop(drop_cols,axis = 1)

# Handling Our Categorial Data
 we have used OneHotEncoder and applied it on 'profession','car_ownership','house_ownership','married' . This combination is selected by repeated test run and best combination is selected

In [10]:
from sklearn.preprocessing import OneHotEncoder
'''extracting encoded data'''
enc=OneHotEncoder(handle_unknown='ignore', sparse=False)
enc_data = pd.DataFrame(enc.fit_transform(df[target_col]))

one_hot_encoded_data = pd.get_dummies(df, columns = target_col)

'''updating our dataset without distroying our raw dataframe'''
df_new = df.join(enc_data)
df_new.columns = df_new.columns.astype('str')
df_new = df_new.drop(target_col,axis = 1)


# Creating our main X and Y as x_data and y_data

In [11]:
'''our y as y_data'''
y_data = df_new['risk_flag']

'''our x as x_data'''
x_data = df_new.drop(['risk_flag'],axis = 1)


# Spliting Our Data for Training and Testing

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_data,y_data,random_state=47,test_size=0.20)

# Scaling Using Standard Scaler
standard scaler is used because  MinMaxScaler is giving low accuracy . everything is selected after multiple test run . To understand our test run you can refer to our data visualisation file

In [13]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

scaled_col= ['income','age']
scaler=StandardScaler()
X_train.loc[:,scaled_col] = scaler.fit_transform(X_train.loc[:,scaled_col])
X_test.loc[:,scaled_col] = scaler.transform(X_test.loc[:,scaled_col])

# K Nearest Neighbors Classifier is Used as Main Model 
In genral banking system knn is used for loan approval testing but capital owned by the person is dominant feature. And many people due to this are unable to approve their loan. To solve this problem we have removed such kind of data and features. 

In [14]:
from sklearn.neighbors import KNeighborsClassifier
clasify = KNeighborsClassifier(n_neighbors=3)
x_help = X_train.copy()
y_help = Y_train.copy()
clasify.fit(x_help,y_help)

KNeighborsClassifier(n_neighbors=3)

In [15]:
y_pred=clasify.predict(x_help)

# Our Main Recommendation and Score Prediction System


## For Score : 
we have used genral probability predicted by our model for loan approval

## For Recommendation:
we have used data which got their loan approval and closely related to user data and gave tips to user for improving his score 

In [16]:
import math
import random

def find_my_nearest_people_help(x_help_array , y_pred_array , data , no_people,index):
    '''this function finds the nearest neighbour in the data set and return list of 
    those people'''
  
    li = []
    count=0
    
    for i in x_help_array[y_pred_array==1]:
        actual_index = index[count]
        d = math.dist(data,i)
        count = count+1
        li.append((d,i,actual_index))
    li.sort(key=lambda a: a[0])
    return li[0:no_people]

def find_my_nearest_people(x_help , y_pred , user_data):
    '''this function finds the nearest neighbour in the data set and return indexing,
    distances of those people'''
    
    x_help_array = np.array(x_help)
    y_pred_array = y_pred
    no_people = 5
    user_data = np.array(user_data)[0]
   
    indexing = x_help.index[y_pred==1]
    li = find_my_nearest_people_help(x_help_array , y_pred_array , user_data , no_people,indexing)
    
    distances = [x[0] for x in li]
    index_list = [x[2] for x in li]
    return index_list,distances

def refine_input(data):
    '''this function takes raw dataframe and encode and scale them and return dataframe'''
    enc_data = pd.DataFrame(enc.transform(data[target_col]))
    one_hot_encoded_data = pd.get_dummies(data, columns = target_col)
    data = data.join(enc_data)
    data.columns = data.columns.astype('str')
    data.loc[:,scaled_col] = scaler.transform(data.loc[:,scaled_col])
    data = data.drop(target_col,axis = 1)
    return data

def enterred_data(info):
    '''this function takes string and return list of values'''
    li = info.split()
    li[0] = int(li[0])
    li[1] = int(li[1])
    li[2] = int(li[2])
    li[8] = int(li[8])
    li[9] = int(li[9])
    columns_li=['income','age','experience','married','house_ownership','car_ownership','profession','state','current_job_years','current_house_years']
    info_update = pd.DataFrame(np.array(li).reshape(1,-1),columns=columns_li)
    return info_update




def take_input_and_predict():
    '''take input from from user and give classification'''
    data_by_user = input()
    data1 = enterred_data(data_by_user)
    print(data1)
    data1 = refine_input(data1)
    if clasify.predict(data1)[0]==0:
        return 0
        
    else:
        return 1
    

def input_only():
    '''take input from from user and give dataFrame'''
    data_by_user = input()
    data1 = enterred_data(data_by_user)
    data1 = refine_input(data1)    
    return data1 



def find_li(x_help_array , y_pred_array , data , no_people,index):
    
  
    li = []
    count=0
    
    for i in x_help_array:
        actual_index = index[count]
        d = math.dist(data,i)
        count = count+1
        li.append((d,i,actual_index))
    li.sort(key=lambda a: a[0])
    return li[0:no_people]

def _score(data):
    scored = clasify.predict_proba(data)[0][1]*100
    if scored<25:
        return "below requirements"
    elif scored>25 and scored<55:
        return "fair"
    elif scored>55 and scored<75:
        return "okay"
    else:
        return "good"
    

def recommend(df):
    target_col_median = ['income','experience']
    target_col_mode = ['married','house_ownership','car_ownership','current_job_years','current_house_years']
    ans= []
    for i in target_col_median:
        
        ans.append(df[[i]].median()[0])
    for i in target_col_mode:
        ans.append(df[i].value_counts().index[0])
    return ans


In [17]:
#6999938 25 3 single rented yes Air_traffic_controller Delhi 3 13

# Main Run Function 

In [18]:
def run():
    data_of_user = input_only()
    value = clasify.predict(data_of_user)[0]
    scored = _score(data_of_user)
    keys,dist =find_my_nearest_people(x_help , y_pred , data_of_user)
    hh = df.loc[keys]
    if value==0:
        return value,recommend(hh),scored
    else:
        return value,"congo! loan approved",scored
    

In [19]:
run()

(1, 'congo! loan approved', 'good')

# Accuracy Check

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
Y_test_pred = clasify.predict(X_test)
Y_train_pred = clasify.predict(X_train)
print(accuracy_score(Y_train_pred, Y_train))
print(accuracy_score(Y_test_pred, Y_test))

0.8813296711035988
0.8440140845070423


In [21]:
df

,income,age,experience,married,house_ownership,car_ownership,profession,state,current_job_years,current_house_years,risk_flag
0,6050608,70,0,single,rented,yes,Comedian,Rajasthan,0,14,1
1,668849,74,15,single,rented,no,Psychologist,Gujarat,13,11,1
2,5979323,56,11,single,rented,no,Firefighter,Madhya_Pradesh,6,13,1
3,7812712,59,14,single,owned,no,Architect,Jharkhand,11,11,0
4,3952093,57,7,single,rented,no,Police_officer,Uttar_Pradesh,7,10,1
...,...,...,...,...,...,...,...,...,...,...,...
70991,2837107,70,3,single,rented,no,Consultant,West_Bengal,3,11,1
70992,5554480,25,19,single,rented,yes,Engineer,Bihar,12,10,0
70993,9678663,46,0,single,rented,no,Surgeon,Andhra_Pradesh,0,12,0
70994,6741252,79,19,married,rented,no,Artist,Karnataka,13,12,0
